##### import library

In [1]:
import pysam
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
from scipy.stats import fisher_exact
from sklearn.metrics import confusion_matrix
# import utility
import importlib
pd.set_option('mode.chained_assignment', None)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


##### function to generate mutation file combined

In [2]:
def parquet_to_csv_converter(input_folder_path, output_folder_path, flank_size, genome, complete_data):

    import os
    import pandas as pd

    counter = 0

    for filename in os.listdir(input_folder_path):

        try:
            if ((os.path.isfile(os.path.join(input_folder_path, filename))==True) & (os.path.join(input_folder_path, filename).endswith(".tsv")==True)):

                # df = pd.read_parquet(os.path.join(input_folder_path, filename))
                counter  = counter + 1
                dfi = pd.read_csv(os.path.join(input_folder_path, filename), sep = "\t", encoding='utf-8')
                
                # print("Converted file is :" + str(counter) + " with file name " + filename)
                # print(dfi.columns)
                df = dfi[["CHROM", "POS", "REF", "ALT", "ANN", 'G70_ref_counts', 'G1410_ref_counts', 'G2640_ref_counts',
       'G5150_ref_counts', 'G7530_ref_counts', 'G10150_ref_counts',
       'G70_alt_counts', 'G1410_alt_counts', 'G2640_alt_counts',
       'G5150_alt_counts', 'G7530_alt_counts', 'G10150_alt_counts']]

                for i in range(df.shape[0]):

                  if ((len(df.loc[i, "REF"])==1) & (len(df.loc[i, "ALT"])==1)):

                    # print("Hi")

                    df.loc[i, "up stream"] = genome.fetch(df.loc[i, "CHROM"], df.loc[i, "POS"]-flank_size-1, df.loc[i, "POS"]-1)
                    df.loc[i, "down stram"] = genome.fetch(df.loc[i, "CHROM"], df.loc[i, "POS"], df.loc[i, "POS"] + flank_size)

                df["ref_flank"] = df["up stream"]+ df["REF"] + df["down stram"]
                df["alt_flank"] = df["up stream"]+ df["ALT"] + df["down stram"]
                # df.to_excel(output_folder_path + os.path.splitext(os.path.basename(os.path.join(input_folder_path, filename)))[0] + ".xlsx", index = False)

                df["file_name"] = filename

                df['population_name'] = os.path.splitext(os.path.basename(os.path.join(input_folder_path, filename)))[0][0:os.path.splitext(os.path.basename(os.path.join(input_folder_path, filename)))[0].find("_")]


                # print("File completed: ", counter)
                complete_data = pd.concat([complete_data, df])



        except:
            continue

        # break
    return complete_data

In [3]:
input_folder_path = "../data/well_output/"
output_folder_path = "../Results/VCF/"

##### Generate flank data

In [4]:
complete_data = pd.DataFrame(columns = ["CHROM", "POS", "REF", "ALT", "ANN", 'G70_ref_counts', 'G1410_ref_counts', 'G2640_ref_counts',
       'G5150_ref_counts', 'G7530_ref_counts', 'G10150_ref_counts',
       'G70_alt_counts', 'G1410_alt_counts', 'G2640_alt_counts',
       'G5150_alt_counts', 'G7530_alt_counts', 'G10150_alt_counts', 'population_name'])

flank_size = 10
kmer_length =10
file_path_yeast = "../data/w303_vlte.fasta"
genome = pysam.FastaFile(file_path_yeast)

complete_data = parquet_to_csv_converter(input_folder_path, output_folder_path, flank_size, genome, complete_data)

OSError: file `../data/w303_vlte.fasta` not found

In [5]:
complete_data

,CHROM,POS,REF,ALT,ANN,G70_ref_counts,G1410_ref_counts,G2640_ref_counts,G5150_ref_counts,G7530_ref_counts,G10150_ref_counts,G70_alt_counts,G1410_alt_counts,G2640_alt_counts,G5150_alt_counts,G7530_alt_counts,G10150_alt_counts,population_name


In [6]:
complete_data.dropna(subset = ["up stream"], inplace=True, axis = 0)
complete_data

KeyError: ['up stream']

In [ ]:
complete_data.reset_index(inplace =True, drop =True)
complete_data

,CHROM,POS,REF,ALT,ANN,G70_ref_counts,G1410_ref_counts,G2640_ref_counts,G5150_ref_counts,G7530_ref_counts,...,G2640_alt_counts,G5150_alt_counts,G7530_alt_counts,G10150_alt_counts,population_name,up stream,down stram,ref_flank,alt_flank,file_name
0,chrI,135,G,A,A|upstream_gene_variant|MODIFIER|YAL069W|YAL06...,170.0,34.0,120.0,59.0,181.0,...,14.0,9.0,0.0,6.0,P1C08,CCATTACCCT,CCTCCACTCG,CCATTACCCTGCCTCCACTCG,CCATTACCCTACCTCCACTCG,P1C08_full.tsv
1,chrI,140,C,T,T|upstream_gene_variant|MODIFIER|YAL069W|YAL06...,168.0,38.0,123.0,60.0,179.0,...,14.0,7.0,0.0,6.0,P1C08,ACCCTGCCTC,ACTCGTTACC,ACCCTGCCTCCACTCGTTACC,ACCCTGCCTCTACTCGTTACC,P1C08_full.tsv
2,chrI,366,A,G,G|synonymous_variant|LOW|YAL069W|YAL069W|trans...,88.0,40.0,81.0,54.0,111.0,...,0.0,0.0,0.0,0.0,P1C08,TGCTTACCCT,CCACTTTATA,TGCTTACCCTACCACTTTATA,TGCTTACCCTGCCACTTTATA,P1C08_full.tsv
3,chrI,374,A,G,G|missense_variant|MODERATE|YAL069W|YAL069W|tr...,94.0,40.0,81.0,55.0,111.0,...,0.0,4.0,0.0,0.0,P1C08,CTACCACTTT,TACCACCACC,CTACCACTTTATACCACCACC,CTACCACTTTGTACCACCACC,P1C08_full.tsv
4,chrI,395,C,G,G|missense_variant|MODERATE|YAL069W|YAL069W|tr...,90.0,45.0,81.0,52.0,121.0,...,0.0,6.0,0.0,0.0,P1C08,ACATGCCATA,TCACCCTCAC,ACATGCCATACTCACCCTCAC,ACATGCCATAGTCACCCTCAC,P1C08_full.tsv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144704,chrMito,69308,A,T,T|intergenic_region|MODIFIER|||||||||||||,51.0,44.0,62.0,54.0,63.0,...,0.0,6.0,0.0,0.0,P2G09,TATTTTCCGT,TATATCTTTA,TATTTTCCGTATATATCTTTA,TATTTTCCGTTTATATCTTTA,P2G09_full.tsv
144705,chrMito,69315,T,C,C|intergenic_region|MODIFIER|||||||||||||,51.0,44.0,62.0,49.0,63.0,...,0.0,6.0,0.0,0.0,P2G09,CGTATATATC,TTAATTTAAT,CGTATATATCTTTAATTTAAT,CGTATATATCCTTAATTTAAT,P2G09_full.tsv
144706,chrMito,71519,A,G,G|intergenic_region|MODIFIER|||||||||||||,90.0,32.0,60.0,39.0,68.0,...,5.0,0.0,6.0,0.0,P2G09,AATTCCTATT,AGGACGATAA,AATTCCTATTAAGGACGATAA,AATTCCTATTGAGGACGATAA,P2G09_full.tsv
144707,chrMito,71520,A,G,G|intergenic_region|MODIFIER|||||||||||||,90.0,29.0,60.0,39.0,67.0,...,5.0,0.0,5.0,0.0,P2G09,ATTCCTATTA,GGACGATAAT,ATTCCTATTAAGGACGATAAT,ATTCCTATTAGGGACGATAAT,P2G09_full.tsv


In [ ]:
import kaos
kmer_length =10
file_path_yeast = "../data/w303_vlte.fasta"
fcgr_sequence_yeast = kaos.kaos_fcgr().chaos_frequency_dictionary(fasta_string= kaos.Read().read_fasta_removing_non_atgc(file_path_yeast), kmer_length=kmer_length, pseudo_count=True)

In [ ]:
import pandas as pd
import numpy as np

rows = []

kmer_length = 10

df_merge_int_snp = complete_data.copy()

count = 0

for i in range(df_merge_int_snp.shape[0]):

    try:

        ref_seq = df_merge_int_snp.loc[i, 'ref_flank']
        alt_seq = df_merge_int_snp.loc[i, 'alt_flank']
        center_pos = int(len(ref_seq) / 2)

        # print(ref_seq, alt_seq)

        # print(i)
        
        if not (set(list(set(list(ref_seq)))).issubset(['G', 'T', 'A', 'C'])):
            continue
        if not (set(list(set(list(alt_seq)))).issubset(['G', 'T', 'A', 'C'])):
            continue     

        for j in range(kmer_length):
            altered_allele = alt_seq[center_pos - kmer_length + j + 1:center_pos + 1 + j]
            reference_allele = ref_seq[center_pos - kmer_length + j + 1:center_pos + j + 1]

            # print(altered_allele, reference_allele)
            # if (fcgr_sequence_yeast[altered_allele]==0):
            #     count = count +1
            rows.append({
                'altered allete': altered_allele,
                'reference allele': reference_allele,
                'gain': np.log(fcgr_sequence_yeast[altered_allele] / fcgr_sequence_yeast[reference_allele]),
                'Position': df_merge_int_snp.loc[i, 'POS'],
                'CHROM': df_merge_int_snp.loc[i, 'CHROM'],
                'REF': df_merge_int_snp.loc[i, 'REF'],
                'ALT': df_merge_int_snp.loc[i, 'ALT'],
                # 'group_mutation_type' : df_merge_int_snp.loc[i, 'group_mutation_type'],
                'population_name' : df_merge_int_snp.loc[i, 'population_name'],
                'G70_ref_counts': df_merge_int_snp.loc[i, 'G70_ref_counts'], 
                'G1410_ref_counts': df_merge_int_snp.loc[i, 'G1410_ref_counts'], 
                'G2640_ref_counts': df_merge_int_snp.loc[i, 'G2640_ref_counts'], 
                'G5150_ref_counts': df_merge_int_snp.loc[i, 'G5150_ref_counts'], 
                'G7530_ref_counts': df_merge_int_snp.loc[i, 'G7530_ref_counts'],
                'G10150_ref_counts': df_merge_int_snp.loc[i, 'G10150_ref_counts'], 
                'G70_alt_counts': df_merge_int_snp.loc[i, 'G70_alt_counts'],
                'G1410_alt_counts': df_merge_int_snp.loc[i, 'G1410_alt_counts'],
                'G2640_alt_counts': df_merge_int_snp.loc[i, 'G2640_alt_counts'],
                'G5150_alt_counts': df_merge_int_snp.loc[i, 'G5150_alt_counts'],
                'G7530_alt_counts': df_merge_int_snp.loc[i, 'G7530_alt_counts'],
                'G10150_alt_counts': df_merge_int_snp.loc[i, 'G10150_alt_counts']
        })
    except:
        continue
            

        
    # del altered_allele, reference_allele, ref_seq, alt_seq, center_pos
        
#     if(i==240000):
#         break


df_complete = pd.DataFrame(rows)



In [ ]:
# complete_data.to_

In [ ]:
key_columns = ["Position", "CHROM", "REF", "ALT", 'population_name']

In [ ]:
df_complete

,altered allete,reference allele,gain,Position,CHROM,REF,ALT,population_name,G70_ref_counts,G1410_ref_counts,G2640_ref_counts,G5150_ref_counts,G7530_ref_counts,G10150_ref_counts,G70_alt_counts,G1410_alt_counts,G2640_alt_counts,G5150_alt_counts,G7530_alt_counts,G10150_alt_counts
0,CATTACCCTA,CATTACCCTG,0.955511,135,chrI,G,A,P1C08,170.0,34.0,120.0,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0
1,ATTACCCTAC,ATTACCCTGC,1.131402,135,chrI,G,A,P1C08,170.0,34.0,120.0,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0
2,TTACCCTACC,TTACCCTGCC,0.716678,135,chrI,G,A,P1C08,170.0,34.0,120.0,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0
3,TACCCTACCT,TACCCTGCCT,0.635989,135,chrI,G,A,P1C08,170.0,34.0,120.0,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0
4,ACCCTACCTC,ACCCTGCCTC,1.178655,135,chrI,G,A,P1C08,170.0,34.0,120.0,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335375,TAATCTATAT,TAATATATAT,-2.197225,71534,chrMito,A,C,P2G09,73.0,19.0,44.0,14.0,50.0,21.0,0.0,1.0,2.0,0.0,3.0,0.0
1335376,AATCTATATA,AATATATATA,-2.297165,71534,chrMito,A,C,P2G09,73.0,19.0,44.0,14.0,50.0,21.0,0.0,1.0,2.0,0.0,3.0,0.0
1335377,ATCTATATAT,ATATATATAT,-3.315454,71534,chrMito,A,C,P2G09,73.0,19.0,44.0,14.0,50.0,21.0,0.0,1.0,2.0,0.0,3.0,0.0
1335378,TCTATATATA,TATATATATA,-3.143530,71534,chrMito,A,C,P2G09,73.0,19.0,44.0,14.0,50.0,21.0,0.0,1.0,2.0,0.0,3.0,0.0


In [ ]:
df_complete[df_complete.duplicated(subset=key_columns)]

,altered allete,reference allele,gain,Position,CHROM,REF,ALT,population_name,G70_ref_counts,G1410_ref_counts,G2640_ref_counts,G5150_ref_counts,G7530_ref_counts,G10150_ref_counts,G70_alt_counts,G1410_alt_counts,G2640_alt_counts,G5150_alt_counts,G7530_alt_counts,G10150_alt_counts
1,ATTACCCTAC,ATTACCCTGC,1.131402,135,chrI,G,A,P1C08,170.0,34.0,120.0,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0
2,TTACCCTACC,TTACCCTGCC,0.716678,135,chrI,G,A,P1C08,170.0,34.0,120.0,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0
3,TACCCTACCT,TACCCTGCCT,0.635989,135,chrI,G,A,P1C08,170.0,34.0,120.0,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0
4,ACCCTACCTC,ACCCTGCCTC,1.178655,135,chrI,G,A,P1C08,170.0,34.0,120.0,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0
5,CCCTACCTCC,CCCTGCCTCC,0.251314,135,chrI,G,A,P1C08,170.0,34.0,120.0,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335375,TAATCTATAT,TAATATATAT,-2.197225,71534,chrMito,A,C,P2G09,73.0,19.0,44.0,14.0,50.0,21.0,0.0,1.0,2.0,0.0,3.0,0.0
1335376,AATCTATATA,AATATATATA,-2.297165,71534,chrMito,A,C,P2G09,73.0,19.0,44.0,14.0,50.0,21.0,0.0,1.0,2.0,0.0,3.0,0.0
1335377,ATCTATATAT,ATATATATAT,-3.315454,71534,chrMito,A,C,P2G09,73.0,19.0,44.0,14.0,50.0,21.0,0.0,1.0,2.0,0.0,3.0,0.0
1335378,TCTATATATA,TATATATATA,-3.143530,71534,chrMito,A,C,P2G09,73.0,19.0,44.0,14.0,50.0,21.0,0.0,1.0,2.0,0.0,3.0,0.0


In [ ]:
grouped_df_with_accumulated = df_complete.groupby(key_columns).agg({'gain':'sum'}).reset_index()
grouped_df_with_accumulated.rename(columns={'gain': 'accumulate_gain'}, inplace=True)
grouped_df_with_accumulated

,Position,CHROM,REF,ALT,population_name,accumulate_gain
0,31,chrVIII,C,A,P3G06,-3.577119
1,34,2-micron,G,T,P1E04,0.392871
2,34,chrII,G,A,P2C06,-4.563850
3,36,chrIX,C,A,P2F11,-11.012777
4,36,chrIX,C,A,P3F11,-11.012777
...,...,...,...,...,...,...
133533,1525964,chrIV,C,T,P3G10,8.835324
133534,1525964,chrIV,C,T,P3G11,8.835324
133535,1525998,chrIV,T,C,P3B07,-14.496006
133536,1525998,chrIV,T,C,P3B10,-14.496006


In [ ]:
df_complete = pd.merge(df_complete, grouped_df_with_accumulated, on=key_columns, how='left')
df_complete

,altered allete,reference allele,gain,Position,CHROM,REF,ALT,population_name,G70_ref_counts,G1410_ref_counts,...,G5150_ref_counts,G7530_ref_counts,G10150_ref_counts,G70_alt_counts,G1410_alt_counts,G2640_alt_counts,G5150_alt_counts,G7530_alt_counts,G10150_alt_counts,accumulate_gain
0,CATTACCCTA,CATTACCCTG,0.955511,135,chrI,G,A,P1C08,170.0,34.0,...,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0,2.968131
1,ATTACCCTAC,ATTACCCTGC,1.131402,135,chrI,G,A,P1C08,170.0,34.0,...,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0,2.968131
2,TTACCCTACC,TTACCCTGCC,0.716678,135,chrI,G,A,P1C08,170.0,34.0,...,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0,2.968131
3,TACCCTACCT,TACCCTGCCT,0.635989,135,chrI,G,A,P1C08,170.0,34.0,...,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0,2.968131
4,ACCCTACCTC,ACCCTGCCTC,1.178655,135,chrI,G,A,P1C08,170.0,34.0,...,59.0,181.0,33.0,0.0,5.0,14.0,9.0,0.0,6.0,2.968131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335375,TAATCTATAT,TAATATATAT,-2.197225,71534,chrMito,A,C,P2G09,73.0,19.0,...,14.0,50.0,21.0,0.0,1.0,2.0,0.0,3.0,0.0,-23.671064
1335376,AATCTATATA,AATATATATA,-2.297165,71534,chrMito,A,C,P2G09,73.0,19.0,...,14.0,50.0,21.0,0.0,1.0,2.0,0.0,3.0,0.0,-23.671064
1335377,ATCTATATAT,ATATATATAT,-3.315454,71534,chrMito,A,C,P2G09,73.0,19.0,...,14.0,50.0,21.0,0.0,1.0,2.0,0.0,3.0,0.0,-23.671064
1335378,TCTATATATA,TATATATATA,-3.143530,71534,chrMito,A,C,P2G09,73.0,19.0,...,14.0,50.0,21.0,0.0,1.0,2.0,0.0,3.0,0.0,-23.671064


In [ ]:
df_complete.columns

Index(['altered allete', 'reference allele', 'gain', 'Position', 'CHROM',
       'REF', 'ALT', 'population_name', 'G70_ref_counts', 'G1410_ref_counts',
       'G2640_ref_counts', 'G5150_ref_counts', 'G7530_ref_counts',
       'G10150_ref_counts', 'G70_alt_counts', 'G1410_alt_counts',
       'G2640_alt_counts', 'G5150_alt_counts', 'G7530_alt_counts',
       'G10150_alt_counts', 'accumulate_gain'],
      dtype='object')

In [ ]:
ag_data  = df_complete.groupby(['Position', 'CHROM',
       'REF', 'ALT', 'population_name', 'G70_ref_counts', 'G1410_ref_counts',
       'G2640_ref_counts', 'G5150_ref_counts', 'G7530_ref_counts',
       'G10150_ref_counts', 'G70_alt_counts', 'G1410_alt_counts',
       'G2640_alt_counts', 'G5150_alt_counts', 'G7530_alt_counts',
       'G10150_alt_counts'])['accumulate_gain'].mean()
ag_data_group_by = ag_data.reset_index()
ag_data_group_by = pd.DataFrame(ag_data)
ag_data_group_by

accumulate_gain
Position CHROM    REF ALT population_name G70_ref_counts G1410_ref_counts G2640_ref_counts G5150_ref_counts G7530_ref_counts G10150_ref_counts G70_alt_counts G1410_alt_counts G2640_alt_counts G5150_alt_counts G7530_alt_counts G10150_alt_counts                 
31       chrVIII  C   A   P3G06           35.0           31.0             59.0             38.0             42.0             56.0              0.0            0.0              0.0              0.0              11.0             0.0                      -3.577119
34       2-micron G   T   P1E04           74.0           104.0            3.0              0.0              0.0              1.0               0.0            217.0            426.0            323.0            156.0            236.0                     0.392871
         chrII    G   A   P2C06           4.0            0.0              1.0              0.0              2.0              1.0               0.0            0.0              0.0              0.0              16.0             12.0                     -4.563850
36       chrIX    C   A   P2F11           24.0           5.0              39.0             2.0              34.0             2.0               0.0            0.0              0.0              0.0              5.0              0.0                     -11.012777
                          P3F11           44.0           1.0              38.0             4.0              18.0             2.0               0.0            0.0              0.0              0.0              9.0              0.0                     -11.012777
...                                                                                                                                                                                                                                                              ...
1525964  chrIV    C   T   P3G10           0.0            0.0              0.0              1.0              2.0              0.0               3.0            5.0              12.0             7.0              18.0             0.0                       8.835324
                          P3G11           2.0            0.0              1.0              1.0              0.0              0.0               0.0            0.0              6.0              0.0              4.0              0.0                       8.835324
1525998  chrIV    T   C   P3B07           0.0            0.0              1.0              1.0              3.0              0.0               2.0            0.0              0.0              1.0              8.0              0.0                     -14.496006
                          P3B10           0.0            0.0              0.0              4.0              2.0              1.0               0.0            0.0              3.0              2.0              0.0              0.0                     -14.496006
1526031  chrIV    A   G   P3D05           0.0            0.0              0.0              2.0              0.0              1.0               0.0            0.0              0.0              0.0              32.0             6.0                      -5.155277

[133538 rows x 1 columns]

In [ ]:
# df_complete[df_complete.duplicated(key_columns)]

In [ ]:
ag_data_group_by.reset_index(inplace=True)
ag_data_group_by

,Position,CHROM,REF,ALT,population_name,G70_ref_counts,G1410_ref_counts,G2640_ref_counts,G5150_ref_counts,G7530_ref_counts,G10150_ref_counts,G70_alt_counts,G1410_alt_counts,G2640_alt_counts,G5150_alt_counts,G7530_alt_counts,G10150_alt_counts,accumulate_gain
0,31,chrVIII,C,A,P3G06,35.0,31.0,59.0,38.0,42.0,56.0,0.0,0.0,0.0,0.0,11.0,0.0,-3.577119
1,34,2-micron,G,T,P1E04,74.0,104.0,3.0,0.0,0.0,1.0,0.0,217.0,426.0,323.0,156.0,236.0,0.392871
2,34,chrII,G,A,P2C06,4.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,16.0,12.0,-4.563850
3,36,chrIX,C,A,P2F11,24.0,5.0,39.0,2.0,34.0,2.0,0.0,0.0,0.0,0.0,5.0,0.0,-11.012777
4,36,chrIX,C,A,P3F11,44.0,1.0,38.0,4.0,18.0,2.0,0.0,0.0,0.0,0.0,9.0,0.0,-11.012777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133533,1525964,chrIV,C,T,P3G10,0.0,0.0,0.0,1.0,2.0,0.0,3.0,5.0,12.0,7.0,18.0,0.0,8.835324
133534,1525964,chrIV,C,T,P3G11,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,6.0,0.0,4.0,0.0,8.835324
133535,1525998,chrIV,T,C,P3B07,0.0,0.0,1.0,1.0,3.0,0.0,2.0,0.0,0.0,1.0,8.0,0.0,-14.496006
133536,1525998,chrIV,T,C,P3B10,0.0,0.0,0.0,4.0,2.0,1.0,0.0,0.0,3.0,2.0,0.0,0.0,-14.496006


In [ ]:
# ag_data_group_by.to_csv("Gain_calulated_unfiltered_vcf.csv")